# Generating Irony data

**Table of contents**:
- Qwen/Qwen2.5-0.5B-Instruct
    1. Baseline
    2. Targeted
    3. Targeted + Linguistic tags
- deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
    1. Baseline
    2. Targeted
    3. Targeted + Linguistic tags

**Results**:
- irony seems a too difficult topic for this models, the "irony" sentences aren't ironic (also the "targeted" prompts case).
- doesn't follow always the instruction and is very sensitive to prompt, but it follows nicely the JSON format.
- ChatGPT gives examples that are actually ironic.

In [19]:
import os

# CHANGE WORKING DIRECTORY TO ROOT
current_dir = os.path.basename(os.getcwd())
if current_dir == "src":
    os.chdir("..") # Move up by 1
elif os.path.basename(os.getcwd()) == "bai-thesis-nlp":  
    pass # If already at root, stay there
else:
    os.chdir("../..") # Move up by 2 otherwise
    
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import re
from src._utils._helpers import log_synthetic_data, response2json, get_response, set_seed, clear_cuda_cache

In [ ]:
# Create the folder to save the synthetic data
folder_name = "synthetic_data/logs"
os.makedirs(folder_name, exist_ok=True)

# file where the logs will be saved
log_file_path = folder_name + "/semevalirony_log.json"
RECREATE_LOG = False
if os.path.exists(log_file_path) and RECREATE_LOG:
    os.remove(log_file_path)

# DEVICE
device = 'cuda:0'

# DATA
data_path = "real_data/train/semevalironytrainAll.csv"
df = pd.read_csv(data_path)
df = df.rename(columns={"1": "text", "2": "label"})
display(df.head())

first_irony = df[df['label'] == 'irony'].iloc[0].loc['text']
first_non_irony = df[df['label'] == 'non irony'].iloc[0].loc['text']

print("\nFirst irony example:")
print(first_irony)

print("\nFirst non-irony example:")
print(first_non_irony)

,0,text,label
0,0,seeing ppl walking w/ crutches makes me really...,irony
1,1,"look for the girl with the broken smile, ask h...",non irony
2,2,Now I remember why I buy books online @user #s...,irony
3,3,@user @user So is he banded from wearing the c...,irony
4,4,Just found out there are Etch A Sketch apps. ...,irony



First irony example:
seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life

First non-irony example:
look for the girl with the broken smile, ask her if she wants to stay while, and she will be loved. 💕🎵


# Qwen/Qwen2.5-0.5B-Instruct
## 1. Baseline

In [21]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda",
    attn_implementation='flash_attention_2',
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# for qwen we use this system prompt:
system_prompt_qwen = "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."

In [22]:
prompt ="""\
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non-irony". \
Produce 5 examples  for "irony" and 5 examples for "non-irony". 

Use this format for the examples:
text: <text>
label: <label>
"""
generated, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 5.36
GENERATED RESPONSE:
I'm sorry, but I can't produce those specific examples since they don't match your criteria of categorizing them as "irony" or "non-irony." However, I can provide some general examples that might help you understand how to detect irony:

### Example 1: Irony (Non-Irony)
Text: The sun was shining brightly.
Label: Irony

This sentence uses hyperbole ("shining brightly") to emphasize the brightness of the sun, which is not ironic because it's meant to be humorous.

### Example 2: Irony (Non-Irony)
Text: She said she would never leave without me.
Label: Irony

In this example, the person says they will always stay with the speaker even if there's no need, implying an obligation rather than a choice.

### Example 3: Irony (Non-Irony)
Text: He said he'd go on vacation next month.
Label: Irony

The statement implies the speaker has a commitment to continue traveling despite the upcoming vacation, suggesting an obligation rather than just a desire.

### Exa

In [23]:
# With example for the two classes
prompt =f"""\
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non-irony". \
Produce exactly 5 examples  for "irony" and 5 examples for "non-irony". 

Use this format for generating the data:
text: {first_irony}
label: irony

text: {first_non_irony}
label: non-irony
"""
generated, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 4.73
GENERATED RESPONSE:
Here are ten examples for detecting irony in written text:

### Irony (5 examples)
1. **Text:** The company's new policy on remote work made me feel like I was living in a dream world.
   - **Label:** Irony

2. **Text:** She said she would come back tomorrow but now that she has returned, she looks more worried than ever.
   - **Label:** Irony

3. **Text:** They were discussing the future plans for the project when suddenly, it dawned on them that they had forgotten something crucial.
   - **Label:** Irony

4. **Text:** He told me he couldn't finish his homework because he was so tired from playing video games all day long.
   - **Label:** Irony

5. **Text:** He promised me he'd help me with any math problems I have coming up tomorrow.
   - **Label:** Non-irony

### Non-Irony (5 examples)
6. **Text:** This restaurant is amazing! The food is delicious and the service is impeccable.
   - **Label:** Non-irony

7. **Text:** It was raining heavily outsid

In [24]:
# act as a linguist and NLP practitioner
prompt =f"""\
You are an expert linguist and NLP practitioner specializing in irony detection. \
Your task is to generate high-quality synthetic examples of ironic and non-ironic text to improve an irony detection model

Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non-irony". \
Produce exactly 5 examples for "irony" and 5 examples for "non-irony". 

Use this format for generating the data:
text: <text>
label: <label>
"""
generated, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 4.95
GENERATED RESPONSE:
Sure! Here are ten examples that I've generated to detect irony in written text:

### Irony Example 1
> "I was so excited about my new job that I couldn't contain myself and burst into tears."

**Label:** Irony

### Irony Example 2
> "The cat chased the mouse but the mouse didn’t care because it knew the cat would eat it."

**Label:** Irony

### Irony Example 3
> "He said he would go on a vacation next week, but he never showed up at all."

**Label:** Irony

### Irony Example 4
> "She said she would finish her homework tomorrow, but she only came back after school."

**Label:** Irony

### Irony Example 5
> "He said he would come to the party, but he forgot his keys."

**Label:** Irony

### Irony Example 6
> "They said they would meet tomorrow, but when they got there, it was already night."

**Label:** Irony

### Irony Example 7
> "She said she would go to the beach, but she decided to stay home."

**Label:** Irony

### Irony Example 8
> "He said he

In [25]:
prompt = f"""\
You are an expert linguist and NLP practitioner specializing in irony detection. \
Your task is to generate **10 high-quality examples** of "irony" and "non-irony" statements, \
with **5 irony** and **5 non-irony** examples across different contexts.

### **Output Format (JSON)**
Return only a valid JSON list in the following structure:

```json
[
    {{"text": "{first_irony}", "label": "irony"}},
    {{"text": "{first_non_irony}", "label": "non-irony"}},
    ...
]
```
"""
response, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 5.2
GENERATED RESPONSE:
Here are ten examples of both irony and non-irony statements, along with their labels:

```json
[
    {
        "text": "The new job offer was great, but I had to take on some extra work that made me feel like a failure.",
        "label": "irony"
    },
    {
        "text": "I just got a promotion at work - it's been such a long time since I've felt this way.",
        "label": "irony"
    },
    {
        "text": "He said he'd finish the project before Christmas, but now he's procrastinating instead.",
        "label": "irony"
    },
    {
        "text": "She always looks so beautiful in her wedding dress, even though she has never worn one before.",
        "label": "non-irony"
    },
    {
        "text": "This movie is terrible - every scene feels like a dream sequence.",
        "label": "irony"
    },
    {
        "text": "They're having a big party tonight, but everyone seems too busy to attend.",
        "label": "irony"
    },
    {
    

In [26]:
synthetic_data = response2json(response)

print("synthetic_data[:2]: ", synthetic_data[:2])

synthetic_data[:2]:  [{'text': 'The new job offer was great, but I had to take on some extra work that made me feel like a failure.', 'label': 'irony'}, {'text': "I just got a promotion at work - it's been such a long time since I've felt this way.", 'label': 'irony'}]


In [27]:
log_synthetic_data(model_name, "baseline", prompt, synthetic_data, delta_t, output_file=log_file_path)

import json
# Load JSON file
with open(log_file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Print the first log entry
print(json.dumps(data[:1], indent=4, ensure_ascii=False))  # Pretty-print the first entry

Logged 9 examples to synthetic_data/logs/semevalirony_log.json. Time taken: 5.20 seconds
[
    {
        "timestamp": "2025-03-12T22:57:44.973919",
        "model": "Qwen/Qwen2.5-0.5B-Instruct",
        "generation_method": "baseline",
        "prompt": "You are an expert linguist and NLP practitioner specializing in irony detection. Your task is to generate **10 high-quality examples** of \"irony\" and \"non-irony\" statements, with **5 irony** and **5 non-irony** examples across different contexts.\n\n### **Output Format (JSON)**\nReturn only a valid JSON list in the following structure:\n\n```json\n[\n    {\"text\": \"seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life\", \"label\": \"irony\"},\n    {\"text\": \"look for the girl with the broken smile, ask her if she wants to stay while, and she will be loved. 💕🎵\", \"label\": \"non-irony\"},\n    ...\n]\n```\n",
        "time_taken_seconds": 5.2,
        "num_examples": 9,
        "generated_examp

In [28]:
# Convert log entries to a DataFrame
df = pd.DataFrame(data)

# Expand the 'generated_examples' column
samples_df = df.explode("generated_examples").reset_index(drop=True)

# Convert 'generated_examples' (which is still a dictionary) into separate columns
samples_df = pd.concat([samples_df.drop(columns=["generated_examples"]), samples_df["generated_examples"].apply(pd.Series)], axis=1)

display(samples_df.head())

,timestamp,model,generation_method,prompt,time_taken_seconds,num_examples,text,label
0,2025-03-12T22:57:44.973919,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,5.2,9,"The new job offer was great, but I had to take...",irony
1,2025-03-12T22:57:44.973919,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,5.2,9,I just got a promotion at work - it's been suc...,irony
2,2025-03-12T22:57:44.973919,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,5.2,9,He said he'd finish the project before Christm...,irony
3,2025-03-12T22:57:44.973919,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,5.2,9,She always looks so beautiful in her wedding d...,non-irony
4,2025-03-12T22:57:44.973919,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,5.2,9,This movie is terrible - every scene feels lik...,irony


## 2. Targeted synthetic data

In [29]:
prompt = f"""\
You are an expert linguist and NLP practitioner. \
List all key linguistic and semantic phenomena that must be covered by a irony detection model to perform accurately. \
Provide concrete examples illustrating each phenomenon. \
"""
response, delta_t_0 = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 13.98
GENERATED RESPONSE:
Certainly! Here is a comprehensive list of key linguistic and semantic phenomena that need to be covered for an ironic detection model:

### Linguistic Phenomena

1. **Word Choice**:
   - **Example**: "The quick brown fox jumps over the lazy dog."
     - This sentence uses hyperbole ("quick" being very fast) and metaphorical language ("jumps over the lazy dog") to convey sarcasm.

2. **Phonological Variation**:
   - **Example**: "She said 'I'm so hungry.'"
     - The word order in this sentence (e.g., "I'm so hungry.") is inverted, which can indicate sarcasm or misinterpretation.

3. **Pronunciation and Spelling**:
   - **Example**: "It's raining cats and dogs outside."
     - The spelling error ("cats and dogs" instead of "cats and dogs") could be a form of irony, suggesting a misunderstanding or exaggeration.

4. **Sentence Structure**:
   - **Example**: "He ate the cake without even knowing it was there."
     - The lack of clarity in the senten

In [30]:
prompt = f"""\
You are an expert linguist and NLP practitioner. \
List all key linguistic and semantic phenomena that must be considered for an irony detection model to perform accurately. \
Provide only the names of these phenomena as a structured list, without explanations. \
For example: sarcasm, negation, pragmatic inference, unexpected contrast. \
Return the list in a simple bullet-point format.\
"""
response, delta_t_0 = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 0.39
GENERATED RESPONSE:
- Irony Detection
- Semantic Analysis
- Contrast Detection
- Pragmatic Inference
- Negation
- Sarcasm


In [31]:
# Replace "- " at the beginning of each line with a comma
csv_string = re.sub(r"^\s*-\s*", "", response, flags=re.MULTILINE)  # Remove bullet points
csv_string = ", ".join(csv_string.strip().split("\n"))  # Join lines with ", "

prompt = f"""\
Generate 10 realistic sentences illustrating irony detection examples involving {csv_string}. Ensure that:
- 5 sentences are **irony**.
- 5 sentences are **non-irony**.

### **Output Format (JSON)**
Return **only** a valid JSON list in the following structure:

```json
[
    {{"text": "{first_irony}", "label": "irony"}},
    {{"text": "{first_non_irony}", "label": "non-irony"}},
    ...
]
```
"""
response, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)
# synthetic_data = response2json(response)
# log_synthetic_data(model_name, "targeted", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 4.33
GENERATED RESPONSE:
```json
[
    {
        "text": "The cat was sleeping on the couch when the dog came over unexpectedly.",
        "label": "irony"
    },
    {
        "text": "He said he would never let his kids go out on their own but they did.",
        "label": "non-irony"
    },
    {
        "text": "She wore a red dress, but it wasn't formal attire.",
        "label": "irony"
    },
    {
        "text": "I bought the book because I thought it might interest him.",
        "label": "non-irony"
    },
    {
        "text": "They were late for work, but the boss said there had been an error.",
        "label": "irony"
    },
    {
        "text": "She is so smart that everyone thinks she's brilliant.",
        "label": "non-irony"
    },
    {
        "text": "The man drove too fast, causing a traffic jam.",
        "label": "irony"
    },
    {
        "text": "He said he was going to visit his grandparents soon, but now he has other plans.",
        "label":

### Targeted synthetic data with Chat-GPT

The topics that need to be covered are generated from Chat-GPT online.

In [32]:
prompt = f"""\
You are an expert linguist and NLP practitioner. \
Generate 10 realistic sentences illustrating irony detection examples involving.
For each example specify the label as either "irony" or "non-irony".

### **Consider the following Phenomena:**
- **Linguistic Phenomena**  
    - Lexical Choice: Unusual or exaggerated word use.  
    - Negation: Statements that negate obvious facts.  
    - Punctuation: Use of exclamation marks, ellipses, or quotes for emphasis.  
    - Syntactic Cues: Unusual or complex sentence structures.  
    - Contrastive Conjunctions: Use of "but," "however," to signal contradiction.  

- **Semantic Phenomena**  
    - Contextual Incongruity: Discrepancy between words and context.  
    - Polarity Reversal: Positive words with negative intent, or vice versa.  
    - Hyperbole & Understatement: Exaggeration or minimization for effect.  
    - Sarcasm: Mocking statements implying the opposite meaning.  

- **Contextual Cues**  
    - World Knowledge: Understanding cultural or situational references.  
    - Speaker Intent: Inferring the true intention behind words.  
    - Discourse Contrast: Contradictions across multiple sentences.  

### **Output Format (JSON)**
Return only a valid JSON list in the following structure:

```json
[
    {{"text": <text>, "label": <label>}},
    ...
]
```
"""
response, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)
synthetic_data = response2json(response)
log_synthetic_data(model_name, "targeted", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 3.87
GENERATED RESPONSE:
```json
[
    {
        "text": "The cat is everywhere.",
        "label": "Irony"
    },
    {
        "text": "I love ice cream, but I don't like chocolate.",
        "label": "Non-irony"
    },
    {
        "text": "She always says she'll be here tomorrow, but she's actually going somewhere else.",
        "label": "Irony"
    },
    {
        "text": "We should eat more vegetables, but we're having too many meals.",
        "label": "Non-irony"
    },
    {
        "text": "He said he would come over tonight, but he didn't.",
        "label": "Irony"
    },
    {
        "text": "This movie is great, but it's not really what I expected.",
        "label": "Non-irony"
    },
    {
        "text": "They said they'd go on vacation next year, but they haven't decided yet.",
        "label": "Irony"
    },
    {
        "text": "I can't believe you did that, but you're still doing it.",
        "label": "Non-irony"
    },
    {
        "text": "He p

## 3. Targeted + Tags linguistic phenomena 

Now we ask to the model also to identify the linguistic phenomena present in the generated sentence.

```
{  
    "text": "Oh, I absolutely adore being stuck in traffic for hours.",   
    "label": "ironic",  
    "phenomena": ["Polarity inversion", "Hyperbole", "Semantic incongruence", "Lexical exaggeration"]
},
```

In [33]:
prompt = f"""\
You are an expert linguist and NLP specialist in sarcasm and irony detection. \
Generate 10 realistic sentences illustrating irony detection examples.\
For each example specify the label as either "irony" or "non-irony". And also list the key phenomena it covers.

### **Consider the following Phenomena:**
- **Linguistic Phenomena**  
    - Lexical Choice: Unusual or exaggerated word use.  
    - Negation: Statements that negate obvious facts.  
    - Punctuation: Use of exclamation marks, ellipses, or quotes for emphasis.  
    - Syntactic Cues: Unusual or complex sentence structures.  
    - Contrastive Conjunctions: Use of "but," "however," to signal contradiction.  

- **Semantic Phenomena**  
    - Contextual Incongruity: Discrepancy between words and context.  
    - Polarity Reversal: Positive words with negative intent, or vice versa.  
    - Hyperbole & Understatement: Exaggeration or minimization for effect.  
    - Sarcasm: Mocking statements implying the opposite meaning.  

- **Contextual Cues**  
    - World Knowledge: Understanding cultural or situational references.  
    - Speaker Intent: Inferring the true intention behind words.  
    - Discourse Contrast: Contradictions across multiple sentences.  

### **Output Format (JSON)**
Return only a valid JSON list in the following structure:

```json
[
    {{"text": <text>, "label": <corresponding label>, "phenomena": ["<phenomenon1>", "<phenomenon2>", ...]}},
    ...
]
```
"""
response, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)
synthetic_data = response2json(response)
log_synthetic_data(model_name, "targeted + linguistic tags", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 5.19
GENERATED RESPONSE:
```json
[
    {
        "text": "The company's CEO announced they would be launching a new product.",
        "label": "non-irony",
        "phenomena": ["Lexical Choice", "Punctuation"]
    },
    {
        "text": "I'm so tired of this boring lecture.",
        "label": "non-irony",
        "phenomena": ["Negation", "Syntactic Cues"]
    },
    {
        "text": "She said she was going on vacation tomorrow.",
        "label": "non-irony",
        "phenomena": ["Negation", "Sarcasm"]
    },
    {
        "text": "He said he couldn't make it to the party.",
        "label": "non-irony",
        "phenomena": ["Negation", "Hyperbole & Understatement"]
    },
    {
        "text": "They will have to work hard to finish the project.",
        "label": "non-irony",
        "phenomena": ["Contextual Incongruity"]
    },
    {
        "text": "It is raining heavily outside.",
        "label": "non-irony",
        "phenomena": ["Contextual Incongruity"]
   

---

# deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B

## 1. Baseline

In [34]:
clear_cuda_cache(model)

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    attn_implementation='flash_attention_2',
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.generation_config.pad_token_id = tokenizer.pad_token_id

# we don't submit a system prompt as suggested in the model card

In [35]:
prompt ="""\
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non-irony". \
Produce 5 examples  for "irony" and 5 examples for "non-irony". 

Use this format for the examples:
text: <text>
label: <label>
"""
generated, delta_t = get_response(prompt, model, tokenizer)

TIME TAKEN: 17.98
GENERATED RESPONSE:
Okay, so the user has asked me to produce 10 examples of irony in written text, categorizing them into "irony" and "non-irony." They want 5 examples each. The format they provided is specific, with each example having a text and a label.

First, I need to make sure I understand what irony is. Irony is when the meaning of a statement is the opposite of its literal meaning. It's often used in humor or sarcastic contexts. So, I need to come up with both types of examples.

For the "irony" category, I should think of statements where the situation or meaning is inverted. Maybe something like a situation that seems contradictory but is actually true, or a statement that implies the opposite of what's said. Examples could include situations where the outcome is contrary to expectations, or where the action is contrary to the situation.

In the "non-irony" category, these should be statements where the meaning is straightforward and doesn't require invers

In [36]:
# With example for the two classes
prompt =f"""\
Your task is to generate high-quality synthetic examples of ironic and non-ironic text to improve an irony detection model. \
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non-irony". \
Produce exactly 5 examples  for "irony" and 5 examples for "non-irony". 

Use this format for generating the data:
text: {first_irony}
label: irony

text: {first_non_irony}
label: non-irony\
"""
generated, delta_t = get_response(prompt, model, tokenizer)

TIME TAKEN: 16.32
GENERATED RESPONSE:
Okay, so I need to generate 10 examples of synthetic text for improving an irony detection model. The user has specified that exactly 5 should be categorized as "irony" and 5 as "non-irony." 

First, I should understand what makes text irony. Irony occurs when there's a contradiction or imbalance of meaning between the actual situation and the expected reaction. So, in irony, the situation and the emotion expressed are opposite. For non-irony, the situation and emotion are consistent or don't create a contradiction.

Looking at the example provided:
"Seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life
label: irony"

This is irony because the person is excited about their walking condition, which is the opposite of the actual situation, where they might have crutches. So, the excitement is ironic because it's a response to their condition.

For non-irony examples, I need to create sentences where the situation and 

In [37]:
# act as a linguist and NLP practitioner
prompt =f"""\
<context>
You are an expert linguist and NLP practitioner specializing in irony detection.
</context>

Your task is to generate high-quality synthetic examples of ironic and non-ironic text to improve an irony detection model. \
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non-irony". \
Produce exactly 5 examples for "irony" and 5 examples for "non-irony". 

Use this format for generating the data:
text: <text>
label: <label>
"""
generated, delta_t = get_response(prompt, model, tokenizer)

TIME TAKEN: 10.27
GENERATED RESPONSE:
Okay, so I need to create 10 examples of irony and non-irony for a model. I'm not super familiar with irony, but I think it's when something is obviously wrong or contradictory. Let me try to come up with some examples.

Starting with irony, maybe something that's clearly wrong. Like, "The sky is blue, but the ground is red." Wait, that doesn't make sense. Maybe I should think of something more direct. How about "She said she saw the movie, but she didn't watch it." That seems ironic because if she saw it, she should have watched it.

For non-irony, I need something that's not contradictory. Maybe a statement that's just a fact. "The sky is blue." That's straightforward. Or "She is a teacher." That's a neutral statement.

I should make sure each example is clear and concise. Let me try to come up with 5 irony examples and 5 non-irony ones. Maybe for irony: something that's obviously wrong or has a clear contradiction. For non-irony: something that'

In [38]:
prompt = f"""\
You are an expert linguist and NLP practitioner specializing in irony detection. \
Your task is to generate **10 high-quality examples** of ironic and non-ironic statements, \
with **5 ironic** and **5 non-ironic** examples across different contexts.

### **Output Format (JSON)**
Return the final result into a valid JSON list in the following structure:

```json
[
    {{"text": "{first_irony}", "label": "ironic"}},
    {{"text": "{first_non_irony}", "label": "non-ironic"}},
    ...
]
```
"""
response, delta_t = get_response(prompt, model, tokenizer)
synthetic_data = response2json(response)
log_synthetic_data(model_name, "baseline", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 19.83
GENERATED RESPONSE:
Okay, so I need to create 10 examples of ironic and non-ironic statements, with exactly 5 of each. The user has already provided an example, so I need to make sure I follow that format. Let me think about how to approach this.

First, I should understand what makes a statement ironic. Irony occurs when the meaning of the words is opposite to what is implied, often due to the nature of the words or the context. So, I need to identify statements where the effect contradicts the intention or where the situation is inverted.

For the ironic examples, I want to look for situations where the words imply something contrary to what is actually meant. Maybe using words that suggest a negative outcome when they shouldn't, or positive when they should be negative, or vice versa.

Let me brainstorm some scenarios where irony is common. For example, "I see a crocodile crossing the road" is ironic because the presence of a crocodile implies danger, which is the 

## 2. Targeted synthetic data


In [39]:
prompt = f"""\
You are an expert linguistics and NLP practitioner. \
List all key linguistic and semantic phenomena that must be considered for an irony detection model to perform accurately. \
Provide only the names of these key concepts as a structured list, without explanations.

Return the list in a simple bullet-point format, each bullet point must start with a dash "-".\
"""

response, delta_t_0 = get_response(prompt, model, tokenizer)

# Extract bullet points
bullets = re.findall(r"- (.+)", response)
bullets = [b.strip() for b in bullets]  # Remove leading/trailing whites

# Convert to CSV string
csv_string = ", ".join(bullets)

TIME TAKEN: 13.69
GENERATED RESPONSE:
Okay, so I need to figure out the key linguistic and semantic phenomena that are important for detecting irony in text. I'm a bit new to this, so I'll start by breaking down what irony is. Irony is when the intended meaning of two statements is opposite, often because of wordplay or ambiguity. Examples include "Why are you here?" and "Why are you leaving?" which are contradictory.

First, I should think about the grammatical aspects. Maybe things like question words, statements, and how they relate to each other. For instance, if someone is asking a question and then says "I'm done," that's irony because the question and response are contradictory.

Then, there's the concept of contradiction. When two statements are opposites. But that's too broad. I need more specific terms. Maybe the relationship between the subject and predicate in each statement. If the subject is the same but the predicate is contradictory, that's a clue.

I also remember some

In [40]:
prompt = f"""\
Generate 10 realistic sentences illustrating irony detection examples involving {csv_string}. Ensure that:
- 5 sentences are **ironic**.
- 5 sentences are **non-ironic**.

### **Output Format (JSON)**
Return **only** a valid JSON list in the following structure:

```json
[
    {{"text": "{first_irony}", "label": "ironic"}},
    {{"text": "{first_non_irony}", "label": "non-ironic"}},
    ...
]
```
"""
response, delta_t = get_response(prompt, model, tokenizer)
# synthetic_data = response2json(response)
# log_synthetic_data(model_name, "targeted", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 14.2
GENERATED RESPONSE:
Okay, so the user has asked me to generate 10 sentences that illustrate different types of irony. They provided a clear structure with 5 ironic and 5 non-ironic sentences. I need to make sure I understand each type of irony they're looking for.

First, I'll break down each category. For ironic sentences, I should focus on situations where the irony is direct and surprising. Maybe something about walking with chutes, as they mentioned. Then, inverted question structures, so I'll need sentences where the question is phrased backwards but still makes sense.

Contrary clauses are sentences where the usual structure is flipped. I'll have to think of scenarios where the opposite of what's stated makes sense. Paradoxical relationships could involve situations where two unrelated actions lead to a surprising conclusion, like a car breaking down and being repaired.

Subject-predicate inversion is a bit tricky. I'll need to switch the subject and predicate in

### Targeted synthetic data with Chat-GPT

In [41]:
prompt = f"""\
You are an expert linguist and NLP practitioner. \
Generate 10 realistic sentences illustrating irony detection examples.\
For each example specify the label as either "irony" or "non-irony".

### **Consider the following Phenomena:**
- **Linguistic Phenomena**  
    - Lexical Choice: Unusual or exaggerated word use.  
    - Negation: Statements that negate obvious facts.  
    - Punctuation: Use of exclamation marks, ellipses, or quotes for emphasis.  
    - Syntactic Cues: Unusual or complex sentence structures.  
    - Contrastive Conjunctions: Use of "but," "however," to signal contradiction.  

- **Semantic Phenomena**  
    - Contextual Incongruity: Discrepancy between words and context.  
    - Polarity Reversal: Positive words with negative intent, or vice versa.  
    - Hyperbole & Understatement: Exaggeration or minimization for effect.  
    - Sarcasm: Mocking statements implying the opposite meaning.  

- **Contextual Cues**  
    - World Knowledge: Understanding cultural or situational references.  
    - Speaker Intent: Inferring the true intention behind words.  
    - Discourse Contrast: Contradictions across multiple sentences.  

### **Output Format (JSON)**
For each example specify the label as either "irony" or "non-irony".
Return only a valid JSON list in the following structure:

```json
[
    {{"text": <text>, "label": <label>}},
    ...
]
```
"""
response, delta_t = get_response(prompt, model, tokenizer)
synthetic_data = response2json(response)
log_synthetic_data(model_name, "targeted", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 19.84
GENERATED RESPONSE:
Okay, so I need to generate 10 realistic sentences that illustrate irony using the provided phenomena. Let me break down the requirements.

First, I should understand the phenomena: linguistic, semantic, and contextual. The user provided four categories for each, so I'll need to pick one category for each example. 

Linguistic Phenomena include unusual word choice, negation, punctuation, complex sentence structures, and contrastive conjunctions. The semantic phenomena are contextual incongruity, polarity reversal, hyperbole, and sarcasm. Contextual cues are world knowledge, speaker intent, and discourse contrast.

I need to create sentences that fit one of these categories. Let's brainstorm a variety of examples.

1. **Contrastive Conjunctions**: Maybe a statement that reverses the obvious. Like saying "I love him, but he loves me" using "but" to contradict. So "I love him, but he loves me. You should never be in his presence." Here, "but" is a con

## 3. Targeted + Tags linguistic phenomena 

In [42]:
prompt = f"""\
You are an expert linguist and NLP specialist in sarcasm and irony detection. \
Generate 10 realistic sentences illustrating irony detection examples.\
For each example specify the label as either "irony" or "non-irony", give examples for both of them. And also list the key phenomena it covers.

### **Consider the following Phenomena:**
- **Linguistic Phenomena**  
    - Lexical Choice: Unusual or exaggerated word use.  
    - Negation: Statements that negate obvious facts.  
    - Punctuation: Use of exclamation marks, ellipses, or quotes for emphasis.  
    - Syntactic Cues: Unusual or complex sentence structures.  
    - Contrastive Conjunctions: Use of "but," "however," to signal contradiction.  

- **Semantic Phenomena**  
    - Contextual Incongruity: Discrepancy between words and context.  
    - Polarity Reversal: Positive words with negative intent, or vice versa.  
    - Hyperbole & Understatement: Exaggeration or minimization for effect.  
    - Sarcasm: Mocking statements implying the opposite meaning.  

- **Contextual Cues**  
    - World Knowledge: Understanding cultural or situational references.  
    - Speaker Intent: Inferring the true intention behind words.  
    - Discourse Contrast: Contradictions across multiple sentences.  

### **Output Format (JSON)**
Return only a valid JSON list in the following structure:

```json
[
    {{"text": <text>, "label": <corresponding label>, "phenomena": ["<phenomenon1>", "<phenomenon2>", ...]}},
    ...
]
```
"""
response, delta_t = get_response(prompt, model, tokenizer)
synthetic_data = response2json(response)
log_synthetic_data(model_name, "targeted + linguistic tags", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 22.79
GENERATED RESPONSE:
Okay, so I need to create 10 examples of irony and non-irony sentences based on the provided phenomena. Let me first make sure I understand the criteria for each category.

Starting with the label "irony." I know irony is when the meaning of the words is opposite to their literal meaning. Examples could involve exaggerated words, negations, complex sentences, or contextually inconsistent statements.

For "non-irony," it's when the irony is subtle or comes from context or word choice that doesn't change the literal meaning. Maybe using a word without the right connotation or a situation where the irony is implied.

Now, looking at the phenomena:

1. **Lexical Choice**: Using unusual words. For irony, maybe a word that's not commonly used but conveys strong emotions.
2. **Negation**: Statements that contradict obvious facts. For example, "I don't know if I'm right."
3. **Punctuation**: Using exclamation marks or ellipses to emphasize. Irony might com

### Example of data generated by Chat-GPT

"text": "Oh, great! Another rainy day for my outdoor event. Just what I needed!",  
"label": "irony",  
"phenomena": ["Punctuation", "Contextual Incongruity", "Sarcasm"]  

"text": "I love waiting in long lines at the grocery store. It’s my favorite pastime!",  
"label": "irony",  
"phenomena": ["Lexical Choice", "Sarcasm"]  

"text": "I’m really excited to spend the weekend doing absolutely nothing.",  
"label": "irony",  
"phenomena": ["Lexical Choice", "Hyperbole", "Sarcasm"]  

"text": "The movie was so bad, I could barely keep my eyes open. A masterpiece, truly.",  
"label": "irony",  
"phenomena": ["Polarity Reversal", "Sarcasm", "Contextual Incongruity"]  

"text": "He’s the best driver I know. He’s never gotten into an accident... oh wait, he has."  
"label": "irony",  
"phenomena": ["Contrastive Conjunctions", "Contextual Incongruity", "Sarcasm"]  

"text": "Sure, because skipping breakfast is such a great idea for energy."  
"label": "irony",  
"phenomena": ["Sarcasm", "Contextual Incongruity"]  

"text": "I’m excited to try this new restaurant tonight. It’s my favorite place to eat.",  
"label": "non-irony",  
"phenomena": ["Lexical Choice"]  

"text": "She finished her work on time, and it was a great accomplishment for the team.",  
"label": "non-irony",  
"phenomena": ["Contextual Incongruity"]  

"text": "Sure, skipping breakfast is a great idea for boosting energy!",  
"label": "irony",  
"phenomena": ["Sarcasm", "Contextual Incongruity"]  

"text": "The meeting went well, and we made great progress on the project.",  
"label": "non-irony",  
"phenomena": ["Lexical Choice"]  